# 沪京深 —— ADLine

In [1]:
# note: run the code, shut the vpn!

import akshare as ak
import datetime
import pandas as pd
import matplotlib.pyplot as plt

# 沪 A 股
stock_sh_a_spot_em_df = ak.stock_sh_a_spot_em()
# print(stock_sh_a_spot_em_df["代码"])

# 深 A 股
stock_sz_a_spot_em_df = ak.stock_sz_a_spot_em()

sh_stocksCode = stock_sh_a_spot_em_df["代码"]
 
# 沪深京 A 股
stock_zh_a_spot_em_df = ak.stock_zh_a_spot_em()

stocksCode = pd.merge(stock_sh_a_spot_em_df["代码"], stock_sz_a_spot_em_df["代码"], how = 'outer')
# stocksCode = stock_sh_a_spot_em_df["代码"]
stocksCode = stocksCode[:].values # delete column name

In [2]:
# -- coding: utf-8 --
# note: run the code, shut the vpn!

# 选用前复权
def upAndDownDaily(code, startDate, endDate): #code, startDate, endDate have to be strings when input
    stock = ak.stock_zh_a_hist(symbol=code, period="daily", start_date=startDate, end_date=endDate, adjust="qfq")
    dates_range = []
    for i in range(len(stock)):
        date_range = []
        date_range.append(stock['日期'][i])
        date_range.append(stock['涨跌幅'][i])
        dates_range.append(date_range)
    
    dict_dates_range = dict(dates_range) # produce dict from array
    return dict_dates_range

endDate = str(datetime.date.today()) # endDate is today
endDate = endDate.replace('-', '') # delete '-' from endDate eg: 2023-04-19
startDate = str(int(endDate[:4]) - 2) + endDate[4:] # startDate from two years ago

upAndDownStock = dict()

# judge the stock at specific day is up(+ 1) or down(- 1), then + 1 or - 1 in the dict(upAndDownStock) on that day
def judgeUpOrDown(upAndDownStock, key, upOrDown): # upOrDown should be 1 or -1
    if key in upAndDownStock:
            upAndDownStock[key] += upOrDown
    else:
        upAndDownStock[key] = 0
        upAndDownStock[key] += upOrDown
            
for code in stocksCode:
    temp = upAndDownDaily(code[0], startDate, endDate)
    keys = temp.keys()
    for key in keys:
        if temp[key] > 0:
            judgeUpOrDown(upAndDownStock, key, 1)
        elif temp[key] < 0:
            judgeUpOrDown(upAndDownStock, key, -1)

df = pd.DataFrame(data=upAndDownStock, index=['upStocks - downStocks'])

df = (df.T)

df.to_excel('up - down stocks.xlsx')

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [ ]:
# In the up - down stocks.xlsx, add 'date' to line[0] column[0]
# 画图

import matplotlib.pyplot as plt
import pandas as pd

datas = pd.read_excel(r"up - down stocks.xlsx")
print(datas['Unnamed: 0'])
dates = list(datas['date'])
data = list(datas['upStocks - downStocks'])

# plt

breakthroughPoint = 0
ADLineData = []

for num in data:
    breakthroughPoint += num
    ADLineData.append(breakthroughPoint)
    
# print(y)
plt.figure(figsize = (100, 100))
plt.title("ADLine", fontsize = 50)
plt.xlabel("Dates", fontsize = 50)
plt.ylabel("Data", fontsize = 50)

plt.plot(dates, ADLineData, marker='o')
plt.xticks(rotation = -30) #斜着展示，角度从逆时针开始算。
for a, b in zip(dates, ADLineData):  
    plt.text(a, b, (a, b), fontsize = 15)
plt.savefig('ADLine.jpg')

In [ ]:
# MLM

import matplotlib.pyplot as plt
import pandas as pd

datas = pd.read_excel(r"up - down stocks.xlsx")
dates = list(datas['date'])
data = list(datas['upStocks - downStocks'])


divisor = 200
tradingDays = 200

# MLM = [] # MLM Data's formula is: two hundred days' data divided by divisor(200)
firstTwoHundredDaysData = 0
MLMDict = dict() # MLM Data's formula is: two hundred days' data divided by divisor(200)

for i in range(tradingDays):
    firstTwoHundredDaysData += data[i]

temp = 200
MLMDict[dates[temp - 1]] = firstTwoHundredDaysData / tradingDays

while temp != len(data):
    twoHundredDaysData = data[temp] - data[temp - 200]
    MLMDict[dates[temp]] = MLMDict[dates[temp - 1]] + (twoHundredDaysData / tradingDays)
    temp += 1

MLMDates = MLMDict.keys()
MLMData = MLMDict.values()

plt.figure(figsize = (100, 100))
plt.title("MLM", fontsize = 50)
plt.xlabel("MLMDates", fontsize = 50)
plt.ylabel("MLMData", fontsize = 50)

plt.plot(MLMDates, MLMData, marker='o')

ax = plt.gca() # 在matplotlib的图中，默认有四个轴，两个横轴和两个竖轴，可以通过ax = plt.gca()方法获取，
               # gca是‘get current axes’的缩写，获取图像的轴，总共有四个轴top、bottom、left和right
ax.spines['top'].set_visible(False)  
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_position(('data',0)) # 设置x坐标轴到y = 0的位置; ’data'，就是按照刻度值来; 
                                             # 'axes'呢？就是根据坐标轴的百分比，数字取值只能是0.0~1.0; 
                                             # 'outward'呢？就是根据距离边缘位置的距离

plt.xticks(rotation = -30, position = (0, -200)) # rotation 斜着展示，角度从逆时针开始算; position 调整日期的位置，让日期的显示远离x轴
for a, b in zip(MLMDates, MLMData):  
    plt.text(a, b, (a, b), fontsize = 15)
plt.savefig('MLM.jpg')

In [3]:
import akshare as ak
import datetime
import pandas as pd
import matplotlib.pyplot as plt

# 沪 A 股
stock_sh_a_spot_em_df = ak.stock_sh_a_spot_em()
# print(stock_sh_a_spot_em_df["代码"])

# 深 A 股
stock_sz_a_spot_em_df = ak.stock_sz_a_spot_em()

sh_stocksCode = stock_sh_a_spot_em_df["代码"]
 
# 沪深京 A 股
stock_zh_a_spot_em_df = ak.stock_zh_a_spot_em()

stocksCode = pd.merge(stock_sh_a_spot_em_df["代码"], stock_sz_a_spot_em_df["代码"], how = 'outer')
# stocksCode = stock_sh_a_spot_em_df["代码"]
stocksCode = stocksCode[:].values # delete column name
stock = ak.stock_zh_a_hist(symbol='605499', period="daily", start_date='20230101', end_date='20230523', adjust="qfq")
print(stock)

            日期      开盘      收盘      最高      最低    成交量          成交额    振幅  \
0   2023-01-03  178.58  172.00  179.00  170.00  13446  231176475.0  5.06   
1   2023-01-04  170.50  171.80  173.00  170.50   3953   68020233.0  1.45   
2   2023-01-05  173.24  175.95  179.98  171.80   5550   98032315.0  4.76   
3   2023-01-06  175.95  176.00  179.50  173.51   5238   92446329.0  3.40   
4   2023-01-09  176.68  175.00  177.50  172.30   5766  100528562.0  2.95   
..         ...     ...     ...     ...     ...    ...          ...   ...   
85  2023-05-15  171.52  174.37  174.80  169.90   6167  106483174.0  2.83   
86  2023-05-16  175.06  175.99  182.61  173.00   8272  147300912.0  5.51   
87  2023-05-17  175.30  177.00  179.85  175.00   5682  100638769.0  2.76   
88  2023-05-18  176.50  178.85  184.00  176.03   9350  168896231.0  4.50   
89  2023-05-19  177.96  182.30  186.00  177.96   8525  155602385.0  4.50   

     涨跌幅   涨跌额   换手率  
0  -3.32 -5.90  1.77  
1  -0.12 -0.20  0.52  
2   2.42  4.15  0.